# Boto3 S3 Resource Examples

## Install Requirements

In [ ]:
import sys
!pip install boto3

## Boto 3 Resource Examples

Boto 3 Resources are a high level object oriented implementation.

### Create the S3 Resource

To connect to Ceph or other S3 API compatible storage, you'll need to specify `endpoint_url` but for AWS, you can omit that parameter

```python
session = boto3.session.Session(aws_access_key_id='',
                                aws_secret_access_key='')

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url='https://s3.amazonaws.com',
    region_name='us-east-1')
)
```

In [ ]:
import os
import boto3
import botocore

key_id = os.environ.get('S3_ACCESS_KEY_ID')
secret_key = os.environ.get('S3_SECRET_ACCESS_KEY')

session = boto3.session.Session(aws_access_key_id=key_id,
                                aws_secret_access_key=secret_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4')
)



### List Buckets

In [ ]:
for bucket in s3_resource.buckets.all():
    print(bucket.name)

### List objects in a single bucket

In [ ]:
# list all files in a bucket
bucket = s3_resource.Bucket('rhods-pilot')
for obj in bucket.objects.all():
    print(obj.key, obj.last_modified)

### List objects in a "subdirectory" prefix

In [ ]:
bucket = s3_resource.Bucket('rhods-pilot')
filter = bucket.objects.filter(Prefix='s3-folder')
file_list = filter.all()
    
for obj in file_list:
    print(obj.key, obj.last_modified)

### Download all files in a bucket

In [ ]:
import os

s3_source_dir = ''
local_dest_dir = os.path.join(os.getcwd(), 'downloaded-bucket')

full_prefix = os.path.join('', s3_source_dir)
bucket = s3_resource.Bucket('rhods-pilot')

for obj in bucket.objects.filter(Prefix=full_prefix).all():
    new_file_path = os.path.join(local_dest_dir, obj.key)
    print(new_file_path)
    
    try:
        os.makedirs(os.path.dirname(new_file_path))
    except OSError:
        pass
    
    if new_file_path.endswith('/'):
        continue
        
    if os.path.exists(new_file_path):
        os.remove(new_file_path)

    bucket.download_file(obj.key, new_file_path)


### Downloaded prefixed files in a "subdirectory"

In [ ]:
import os


s3_source_dir = 's3-folder'
local_dest_dir = os.path.join(os.getcwd(), 'downloaded-folder')

bucket = s3_resource.Bucket('rhods-pilot')

for obj in bucket.objects.filter(Prefix=s3_source_dir).all():
    stripped_key = obj.key.replace(s3_source_dir, '')
    if stripped_key.startswith('/'):
        stripped_key = stripped_key[1:]
    new_file_path = os.path.join(local_dest_dir, stripped_key)    
    print(new_file_path)
    
    try:
        os.makedirs(os.path.dirname(new_file_path))
    except OSError:
        pass
    
    if new_file_path.endswith('/'):
        continue
        
    if os.path.exists(new_file_path):
        os.remove(new_file_path)

    bucket.download_file(obj.key, new_file_path)

